In [3]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2Config, TFGPT2Model

RuntimeError: Failed to import transformers.models.gpt2.modeling_tf_gpt2 because of the following error (look up to see its traceback):
cannot import name 'keras' from partially initialized module 'keras.api._v2' (most likely due to a circular import) (C:\Users\asmab\OneDrive\Bureau\5IABDS1\nlp\projet\Recipe_generator-1\env\lib\site-packages\keras\api\_v2\__init__.py)

In [1]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', lang='fr')

NameError: name 'GPT2Tokenizer' is not defined

In [ ]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","cleaned_data.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [ ]:
data.sample(5)

In [ ]:
config = GPT2Config.from_pretrained('gpt2',n_layer=6)
config.output_hidden_states = False
model = TFGPT2Model.from_pretrained('gpt2',config=config)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', lang='fr')
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
max_length = 768
sequences = tokenizer.batch_encode_plus(data['recettes'].to_list(), padding=True, truncation=True, return_tensors='tf', max_length=max_length)

# num_classes = len(np.unique(y))
# y = tf.keras.utils.to_categorical(y, num_classes)

In [ ]:
# Split the data into training and testing sets

X = sequences['input_ids'].numpy()
y = sequences['attention_mask'].numpy()
input_dim = X.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

In [ ]:
config = GPT2Config.from_pretrained('gpt2')
config.output_hidden_states = False
model = TFGPT2Model.from_pretrained('gpt2',config=config)
# model.config = config
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(
  loss=loss,
  optimizer=optimizer,
  metrics=metric
)

# Train the model
history = model.fit(
  X_train,
  y_train,
  batch_size=128,
  epochs=10,
  validation_data=(X_test, y_test)
)
